### Prediction des prix de pétroles

#### Objectif commercial
Dans le contexte de l'ensemble de données mondiales sur la production, les prix, les exportations et les exportations nettes de pétrole et de gaz naturel, les données sur la production et les prix du pétrole couvrent la période 1932-2014 (les données de 2014 sont incomplètes) ; la production et les prix du gaz couvrent la période 1955-2014 ; les données sur les exportations et les exportations nettes couvrent la période 1986-2013, nous avons besoin d'analyser les prix historiques et de prédire les prix futurs.


#### Import/export et chargement


In [11]:
import os
import sys
sys.path.append(os.path.join('..'))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [12]:
train_file = os.path.join('.','Data','train.csv') 
test_file = os.path.join('.','Data','test.csv') 
pd_train = pd.read_csv(train_file)
pd_test = pd.read_csv(test_file)

#### Description des données

In [13]:
pd_train.head()
# Que faire des données manquantes ? 

,Unnamed: 0,cty_name,iso3numeric,id,year,eiacty,oil_prod32_14,oil_price_2000,oil_price_nom,oil_value_nom,...,net_gas_exports_mboe,net_gas_exports_value,net_gas_exports_valuePOP,net_oil_gas_exports_valuePOP,population,pop_maddison,sovereign,mult_nom_2000,mult_nom_2014,mult_2000_2014
0,1,Thailand,764,THA,1957,Thailand,NaN,11.476280,1.900000,NaN,...,NaN,NaN,NaN,NaN,25041917.0,25041917.0,1,6.111378,8.401761,1.374774
1,2,Thailand,764,THA,1984,Thailand,788000.0,45.818440,28.780000,1.662344e+08,...,NaN,NaN,NaN,NaN,49769891.0,50533778.0,1,1.657363,2.278499,1.374774
2,3,Slovak Republic,703,SVK,2014,Slovakia,9960.0,71.972588,98.946008,7.223731e+06,...,NaN,NaN,NaN,NaN,5418506.0,NaN,1,0.727393,1.000000,1.374774
3,4,Moldova,498,MDA,1967,Moldova,NaN,9.161955,1.800000,NaN,...,NaN,NaN,NaN,NaN,3452263.0,3452263.0,0,5.153946,7.085508,1.374774
4,5,Vietnam,704,VNM,1996,Vietnam,8850000.0,22.484490,20.668488,1.340775e+09,...,0.0,0.0,0.0,18.589735,74360136.0,74941175.0,1,1.097514,1.508834,1.374774



city_name

Noms des pays dont les données ont été extraites.

iso3numeric

Codes de pays numériques à trois chiffres définis dans l'ISO 3166-1, partie de la norme ISO 3166 publiée par l'Organisation internationale de normalisation (ISO), pour représenter les pays, les territoires dépendants et les zones spéciales d'intérêt géographique. Afghanistan- 004.

id

Même chose que iso3numeric.

Year

Données collectées pour l'année.

eiacity

Nom du pays de l'Energy Information Administration.

oil_prod32_14

Production de pétrole de 1932 à 2014. Données converties de barils en tonnes métriques.

oil_price_2000

Prix nominal du pétrole de chaque année déflaté du prix du pétrole de l'année 2000.

oil_price_nom

dollars américains par baril pour les années respectives.

oil_value_nom

Obtenu en multipliant la production de pétrole et le prix nominal du pétrole pour les années respectives : Le prix nominal du pétrole en Argentine pour l'année était de 0,87 et 1794945 tonnes métriques de pétrole ont été produites. 1Tonne métrique = 7,33 barils Pour 13156946,85 barils, la valeur est 11446543,7595.

oil_value_2000

Production de pétrole (barils) multipliée par le prix nominal multiplié par le multiplicateur d'inflation (mult_nom_2000) pour les années respectives. Exemple : Inde en 1932 8829043.64* 0.87* 12.599874 = 96783008.5419

oil_value_2014

Production de pétrole (barils) multipliée par le prix nominal multiplié par le multiplicateur d'inflation (mult_nom_2014) pour les années respectives. Exemple : Australie en 1948 1004,21* 9,82691136* 1,99 = 19637,8824871

gas_prod55_14

Production de gaz de 1955 à 2014.

gas_price_2000_mboe

Prix du gaz en millions de barils équivalent pétrole en 2000. Le pétrole est mesuré en barils et le gaz naturel en pieds cubes. Pour faciliter les comparaisons, l'industrie a normalisé la production de gaz naturel en « équivalents barils » de pétrole.

gas_price_2000

Prix du gaz en 2000

gas_price_nom

Prix du gaz pour les années respectives.

gas_value_nom

Valeur du pétrole en multipliant la production de pétrole et le prix nominal du pétrole pour les années respectives.

gas_value_2000

Valeur du gaz en 2000.

gas_value_2014

La valeur du gaz en 2014 tient compte de l'inflation.

oil_gas_value_nom

Valeur du gaz et du pétrole produits pour les années respectives. Exemple : La valeur du gaz pour l'Albanie en 1934 est de 10012,78 et la valeur du gaz est de 0 (on suppose que c'est zéro car les données sont vides). Valeur totale du pétrole = 10012,78+0 = 10012,78

oil_gas_value_2000

Valeur du gaz et du pétrole en 2000, corrigée de l'inflation.

oil_gas_value_2014

Valeur du pétrole et du gaz en 2014.

oil_gas_valuePOP_nom

Il s'agit du revenu pétrolier par habitant, qui mesure la valeur annuelle, en termes réels, de la production de pétrole et de gaz naturel d'un pays, divisée par sa population. Exemple : Pour l'Albanie en 1934, la valeur du pétrole par habitant est la suivante valeur du pétrole par habitant/population = 10012,78/1003000 = 0,009983.

oil_gas_valuePOP_2000

Même chose que oil_gas_valuePOP_nom.

oil_gas_valuePOP_2014

Même chose que oil_gas_valuePOP_nom.

oil_exports

Données sur les exportations de pétrole des pays pour les années respectives. Les exportations nettes sont calculées en soustrayant les importations.

net_oil_exports

Les exportations nettes de pétrole sont la valeur des exportations totales de pétrole d'un pays moins la valeur de ses importations totales de pétrole. Les valeurs négatives indiquent que les exportations sont inférieures aux importations, ce qui crée une balance commerciale négative.

net_oil_exports_mt

Exportations nettes de pétrole en tonnes métriques par an.

net_oil_exports_value

Valeur des exportations nettes de pétrole produites au cours de l'année concernée.

net_oil_exports_valuePOP

Valeur des exportations nettes de pétrole par habitant, Valeur des exportations nettes de pétrole d'un pays divisée par sa population.

gas_exports

Exportations de gaz pour les pays pour les années respectives en tonnes métriques par an.

net_gas_exports_bcf

Exportations nettes de gaz en milliards de pieds cubes par jour.

net_gas_exports_mboe

Exportations nettes de gaz en millions de barils équivalent pétrole.

net_gas_exports_value

Valeur totale des exportations nettes de gaz par pays pour les années respectives.

net_gas_exports_valuePOP

Valeur des exportations nettes de gaz par habitant, Valeur des exportations nettes de gaz des exportations de pétrole d'un pays divisée par sa population.

net_oil_gas_exports_valuePOP

Valeur des exportations nettes de pétrole et de gaz par habitant, Valeur des exportations nettes de pétrole et de gaz d'un pays divisée par sa population.

population

Les données sur la population sont tirées des indicateurs de développement mondial de la Banque mondiale.

pop_maddison

Les données sur la population sont tirées de la base de données d'Angus Maddison pour les années antérieures à 1950.

sovereign

Des données ont été collectées pour les pays avant qu'ils ne deviennent souverains. Indiqué par une variable binaire. 0- Non souverain et 1- souverain.

mult_nom_2000

Multiplicateur d'inflation pour passer de la valeur nominale à la valeur constante de 2000 en dollars américains.

mult_nom_2014

Multiplicateur d'inflation pour convertir la valeur nominale en valeur constante pour 2014 en dollars américains.

mult_2000_2014

Multiplicateur d'inflation pour convertir les valeurs nominales en valeurs constantes de 2000 et de 2014 en dollars américains.

#### Évaluation
La mesure d'évaluation utilisée dans ce concours est le RMSE (Root Mean Squared Error).